In [1]:
%load_ext dotenv
%dotenv -o

import os
not not os.environ["POLYGON_API_KEY"]

True

In [2]:
from zipline_polygon_bundle.config import PolygonConfig
from zipline_polygon_bundle import date_to_path, trades_schema, cast_trades, custom_aggs_schema, custom_aggs_partitioning, generate_csv_trades_tables, trades_to_custom_aggs

from typing import Iterator, Tuple

import pandas as pd

import pyarrow as pa
from pyarrow import dataset as pa_ds
from pyarrow import compute as pa_compute
from pyarrow import parquet as pa_parquet

from fsspec.implementations.arrow import ArrowFSWrapper

import os

import datetime
import pandas_market_calendars
import numpy as np

import resource

from itables import show

In [3]:
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

417616

In [4]:
resource.getrusage(resource.RUSAGE_SELF)

resource.struct_rusage(ru_utime=3.373119, ru_stime=1.391889, ru_maxrss=417616, ru_ixrss=0, ru_idrss=0, ru_isrss=0, ru_minflt=68637, ru_majflt=0, ru_nswap=0, ru_inblock=116832, ru_oublock=312, ru_msgsnd=0, ru_msgrcv=0, ru_nsignals=0, ru_nvcsw=4679, ru_nivcsw=471)

In [5]:
if not "CUSTOM_ASSET_FILES_DIR" in os.environ:
    os.environ["CUSTOM_ASSET_FILES_DIR"]= "/home/jovyan/data"

config = PolygonConfig(
    environ=os.environ,
    calendar_name="NYSE",
    start_date="2021-01-01",
    end_date="2021-01-08",
    agg_time="1min",
)

pd.set_option('display.width', 250)

print(f"{config.minute_aggs_dir=}")
print(f"{config.trades_dir=}")
print(f"{config.custom_aggs_dir=}")
print(f"{config.aggs_dir=}")
print(f"{config.by_ticker_dir=}")

config.minute_aggs_dir='/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/minute_aggs_v1'
config.trades_dir='/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1'
config.custom_aggs_dir='/home/jovyan/data/60sec_aggs'
config.aggs_dir='/home/jovyan/data/60sec_aggs'
config.by_ticker_dir='/home/jovyan/data/60sec_aggs_by_ticker'


In [4]:
aggs_ds = pa_ds.dataset(config.custom_aggs_dir, schema=custom_aggs_schema(), partitioning=custom_aggs_partitioning())
aggs_ds.schema

ticker: string not null
volume: int64 not null
open: double not null
close: double not null
high: double not null
low: double not null
window_start: timestamp[ns, tz=UTC] not null
transactions: int64 not null
date: date32[day] not null
year: uint16 not null
month: uint8 not null

In [6]:
np.full(10, datetime.date(2020, 10, 20))

array([datetime.date(2020, 10, 20), datetime.date(2020, 10, 20),
       datetime.date(2020, 10, 20), datetime.date(2020, 10, 20),
       datetime.date(2020, 10, 20), datetime.date(2020, 10, 20),
       datetime.date(2020, 10, 20), datetime.date(2020, 10, 20),
       datetime.date(2020, 10, 20), datetime.date(2020, 10, 20)],
      dtype=object)

In [7]:
pa.array(np.full(10, datetime.date(2020, 10, 20)))

[
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20,
  2020-10-20
]

In [5]:
pa_ds.get_partition_keys(aggs_ds.partition_expression)

{}

In [9]:
for fragment in aggs_ds.get_fragments():
    print(f"{fragment.partition_expression=}")
    print(f"{pa_ds.get_partition_keys(fragment.partition_expression)=}")
    # print(f"{fragment.partition_expression.get_partition_values()=}")
    # print(f"{fragment.physical_schema=}")


fragment.partition_expression=<pyarrow.compute.Expression (((year == 2017) and (month == 1)) and (date == 2017-01-03))>
pa_ds.get_partition_keys(fragment.partition_expression)={'date': datetime.date(2017, 1, 3), 'month': 1, 'year': 2017}
fragment.partition_expression=<pyarrow.compute.Expression (((year == 2017) and (month == 1)) and (date == 2017-01-04))>
pa_ds.get_partition_keys(fragment.partition_expression)={'date': datetime.date(2017, 1, 4), 'month': 1, 'year': 2017}
fragment.partition_expression=<pyarrow.compute.Expression (((year == 2017) and (month == 1)) and (date == 2017-01-05))>
pa_ds.get_partition_keys(fragment.partition_expression)={'date': datetime.date(2017, 1, 5), 'month': 1, 'year': 2017}
fragment.partition_expression=<pyarrow.compute.Expression (((year == 2017) and (month == 1)) and (date == 2017-01-06))>
pa_ds.get_partition_keys(fragment.partition_expression)={'date': datetime.date(2017, 1, 6), 'month': 1, 'year': 2017}


In [19]:
for fragment in aggs_ds.get_fragments():
    print(f"{fragment.partition_expression=}")
    print(f"{pa_ds.get_partition_keys(fragment.partition_expression)=}")
    # print(f"{fragment.physical_schema=}")
    table = fragment.to_table()
    # print(f"{table.slice(length=1)=}")
    # print(f"{table.schema=}")
    # print(f"{table=}")
    df = table.to_pandas()
    df['window_start'] = df['window_start'].dt.tz_convert('EST')
    df.info()
    print(f"{df['window_start'].min()=}")
    print(f"{df['window_start'].max()=}")
    print(df)
    print()

fragment.partition_expression=<pyarrow.compute.Expression (((year == 2017) and (month == 1)) and (date == 2017-01-03))>
pa_ds.get_partition_keys(fragment.partition_expression)={'date': datetime.date(2017, 1, 3), 'month': 1, 'year': 2017}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1545399 entries, 0 to 1545398
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   ticker        1545399 non-null  object             
 1   volume        1545399 non-null  int64              
 2   open          1545399 non-null  float64            
 3   close         1545399 non-null  float64            
 4   high          1545399 non-null  float64            
 5   low           1545399 non-null  float64            
 6   window_start  1545399 non-null  datetime64[ns, EST]
 7   transactions  1545399 non-null  int64              
dtypes: datetime64[ns, EST](1), float64(4), int64(2), object(1)
memory usage: 94

In [6]:
table_gen = generate_csv_trades_tables(config)

In [7]:
date, trades_table = next(table_gen)
date, trades_table

(datetime.date(2021, 1, 4),
 pyarrow.Table
 ticker: string not null
 conditions: string not null
 correction: string not null
 exchange: int8 not null
 id: string not null
 participant_timestamp: timestamp[ns, tz=UTC] not null
 price: double not null
 sequence_number: int64 not null
 sip_timestamp: timestamp[ns, tz=UTC] not null
 size: int64 not null
 tape: int8 not null
 trf_id: int64 not null
 trf_timestamp: timestamp[ns, tz=UTC] not null
 ----
 ticker: [["A","A","A","A","A",...,"A","A","A","A","A"],["A","A","A","A","A",...,"AA","AA","AA","AA","AA"],...,["ZVO","ZVO","ZVO","ZVO","ZVO",...,"ZYNE","ZYNE","ZYNE","ZYNE","ZYNE"],["ZYNE","ZYNE","ZYNE","ZYNE","ZYNE",...,"ZYXI","ZYXI","ZYXI","ZYXI","ZYXI"]]
 conditions: [["12,37","12,37","12,37","12,37","12,37",...,"37","14,41","14,37,41","14,41","14,37,41"],["14,37,41","14,37,41","14,37,41","14,37,41","14,41",...,"","","14,41","14,41","14,37,41"],...,["37","37","","","37",...,"37","37","37","14,37,41","37"],["","37","37","37","37",...,"12,37

In [8]:
aggs_table = trades_to_custom_aggs(config, date, trades_table)
aggs_table

date=datetime.date(2021, 1, 4) pa.total_allocated_bytes()=6924336192
mp.backend_name='mimalloc' mp=<pyarrow.MemoryPool backend_name=mimalloc bytes_allocated=6924336192 max_memory=6931781696>
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss=7806996


pyarrow.Table
ticker: string not null
window_start: timestamp[ns, tz=UTC]
open: double
high: double
low: double
close: double
size_sum: int64
transactions: int64
----
ticker: [["CTEST.A","MTEST.A","PTEST.A","ATEST.A","CTEST.A",...,"ZOM","ZROZ","ZSL","ZTEST","ZTST"]]
window_start: [[2021-01-04 06:30:00.000000000Z,2021-01-04 06:30:00.000000000Z,2021-01-04 06:30:00.000000000Z,2021-01-04 06:31:00.000000000Z,2021-01-04 06:31:00.000000000Z,...,2021-01-05 01:00:00.000000000Z,2021-01-05 01:00:00.000000000Z,2021-01-05 01:00:00.000000000Z,2021-01-05 01:00:00.000000000Z,2021-01-05 01:00:00.000000000Z]]
open: [[10.604894,14.720926,10.593358,12.645032,14.515077,...,0.35,162.67,6.41,10.1,40.17]]
high: [[11.510764,14.720926,12.273018,14.552261,14.515077,...,0.35,162.67,6.41,10.1,40.17]]
low: [[10.604894,14.720926,10.593358,10.854299,11.478276,...,0.35,162.67,6.41,10.1,40.17]]
close: [[11.510764,14.720926,12.273018,10.854299,11.556036,...,0.35,162.67,6.41,10.1,40.17]]
size_sum: [[222,111,222,666,444,.

In [10]:
aggs_df = aggs_table.slice(length=1000).to_pandas()
show(aggs_df)

In [10]:
len(trades_table)

76084591

In [12]:
tt = trades_table.append_column('foo', pa.array([123] * len(trades_table)))
tt

pyarrow.Table
ticker: string not null
conditions: string not null
correction: string not null
exchange: int8 not null
id: string not null
participant_timestamp: timestamp[ns, tz=UTC] not null
price: double not null
sequence_number: int64 not null
sip_timestamp: timestamp[ns, tz=UTC] not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: timestamp[ns, tz=UTC] not null
foo: int64
----
ticker: [["A","A","A","A","A",...,"A","A","A","A","A"],["A","A","A","A","A",...,"AA","AA","AA","AA","AA"],...,["ZVO","ZVO","ZVO","ZVO","ZVO",...,"ZYNE","ZYNE","ZYNE","ZYNE","ZYNE"],["ZYNE","ZYNE","ZYNE","ZYNE","ZYNE",...,"ZYXI","ZYXI","ZYXI","ZYXI","ZYXI"]]
conditions: [["12,37","12,37","12,37","12,37","12,37",...,"37","14,41","14,37,41","14,41","14,37,41"],["14,37,41","14,37,41","14,37,41","14,37,41","14,41",...,"","","14,41","14,41","14,37,41"],...,["37","37","","","37",...,"37","37","37","14,37,41","37"],["","37","37","37","37",...,"12,37","12,37","12,37","12,37","12,37"]